### SJM-BL Simulation Study with Parallel Monte Carlo

This code runs multiple Monte Carlo simulations of the 1-state, 2-state, and 3-state processes, computes performance metrics for each run, and then uses a Wilcoxon test to compare SJM-BL against all other strategies.

#### 1.0 Loading packages

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
from joblib import Parallel, delayed
import multiprocessing

# Hidden Markov Model utilities
from hmmlearn.hmm import GaussianHMM
from sklearn.cluster import KMeans

# PyPortfolioOpt
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models

# Sparse Jump Model utilities
from jumpmodels.sparse_jump import SparseJumpModel
from jumpmodels.preprocess import StandardScalerPD, DataClipperStd

### 2.0 Data Simulation

#### 2.1 Simulating the 1-state data
We are simulating 6 fictional assets which are representing the 6 factors in our framework
- **1-State:** A single regime with Student‑t returns.
- **2-State:** A two-regime (bull/bear) HMM with state-dependent parameters.
- **3-State:** A three-regime HMM with specified means and volatilitie

All assets have the same expected return and volatility.

In [22]:
ASSETS = ["Value", "Growth", "LowVol", "Size", "Momentum", "Quality"]
N_ASSETS = len(ASSETS)
CONST_RET = 0.000461  # Hypothetical constant daily return used
RISK_FREE_RATE = 0.02 / 252
TRANSACTION_COST = 0.0005
BL_TAU = 0.1  # Black-Litterman tau parameter

In [23]:
def simulate_1state_data(num_days, seed=None):
    """
    Simulate a single-state (Student-t) process for all assets.
    Each call uses its own RNG to ensure new draws if seed changes.
    """
    local_rng = np.random.default_rng(seed)

    mu = 0.000461
    sig = 0.008388
    dof = 5

    corr = np.full((N_ASSETS, N_ASSETS), 0.185)
    np.fill_diagonal(corr, 1.0)
    cov = np.outer(np.full(N_ASSETS, sig), np.full(N_ASSETS, sig)) * corr

    z = local_rng.multivariate_normal(mean=np.zeros(N_ASSETS), cov=cov, size=num_days)
    chi = local_rng.chisquare(dof, size=num_days)
    factor = np.sqrt(dof / chi)[:, None]  # scaling for t distribution
    rets = mu + z * factor
    return pd.DataFrame(rets, columns=ASSETS)

#### 2.2 Simulating 2-state data

This function simulates a 2-state HMM (bull/bear) with state‐dependent Student‑t returns.

In [24]:
def simulate_2state_data(num_days, seed=None):
    """
    2-state HMM-like simulation. We keep local_rng to vary the data each time.
    """
    local_rng = np.random.default_rng(seed)

    # Transition matrix
    transmat = np.array([
        [0.9976, 0.0024],
        [0.0232, 0.9768]
    ])
    states = np.zeros(num_days, dtype=int)
    states[0] = local_rng.integers(2)

    for t in range(1, num_days):
        states[t] = local_rng.choice(2, p=transmat[states[t - 1]])

    mu_dict = {0: 0.0006, 1: -0.000881}
    sig_dict = {0: 0.00757, 1: 0.0163}
    corr = np.full((N_ASSETS, N_ASSETS), 0.185)
    np.fill_diagonal(corr, 1.0)

    rets = np.zeros((num_days, N_ASSETS))
    dof = 5
    for t in range(num_days):
        s = states[t]
        mu_s = np.full(N_ASSETS, mu_dict[s])
        sig_s = np.full(N_ASSETS, sig_dict[s])
        cov_s = np.outer(sig_s, sig_s) * corr

        z = local_rng.multivariate_normal(mean=np.zeros(N_ASSETS), cov=cov_s)
        chi = local_rng.chisquare(dof)
        factor = np.sqrt(dof / chi)
        rets[t] = mu_s + factor * z

    return pd.DataFrame(rets, columns=ASSETS), states

#### 2.3 Simulating 3-state data

We are simulating 6 fictional assets which are representing the 6 factors in our framework

In [25]:
def simulate_3state_data(num_days, seed=None):
    """
    3-state HMM-like simulation. Each call uses local_rng so that different seeds
    produce different data.
    """
    local_rng = np.random.default_rng(seed)

    transmat = np.array([
        [0.9989, 0.0004, 0.0007],
        [0.0089, 0.9904, 0.0007],
        [0.0089, 0.0004, 0.9907]
    ])
    states = np.zeros(num_days, dtype=int)
    states[0] = local_rng.integers(3)

    for t in range(1, num_days):
        states[t] = local_rng.choice(3, p=transmat[states[t - 1]])

    mu_list = [0.0008, 0.0, -0.003586]
    sig_list = [0.0070, 0.0050, 0.01897]
    corr = np.full((N_ASSETS, N_ASSETS), 0.185)
    np.fill_diagonal(corr, 1.0)

    rets = np.zeros((num_days, N_ASSETS))
    dof = 5
    for t in range(num_days):
        s = states[t]
        mu_s = np.full(N_ASSETS, mu_list[s])
        sig_s = np.full(N_ASSETS, sig_list[s])
        cov_s = np.outer(sig_s, sig_s) * corr

        z = local_rng.multivariate_normal(mean=np.zeros(N_ASSETS), cov=cov_s)
        chi = local_rng.chisquare(dof)
        factor = np.sqrt(dof / chi)
        rets[t] = mu_s + factor * z

    return pd.DataFrame(rets, columns=ASSETS), states

### 3.0 Training Regime Models

#### 3.1 Training HMM using kmeans clustering initialization

In [26]:
def run_mle(observations, n_components=2, init_type='default', seed=None):
    model = GaussianHMM(n_components=n_components, covariance_type='diag',
                        n_iter=100, random_state=seed)

    if init_type == 'default':
        model.startprob_ = np.array([1.0, 0.0])
        model.transmat_ = np.array([[0.9, 0.1],
                                    [0.1, 0.9]])
        model.means_ = np.zeros((n_components, observations.shape[1]))
        model.covars_ = np.full((n_components, observations.shape[1]), 1e-2)
        # Disable re-initialization of parameters
        model.init_params = ''
    elif init_type == 'kmeans':
        km = KMeans(n_clusters=n_components, n_init=10, random_state=seed)
        labels = km.fit_predict(observations)
        means, covars = [], []
        for i in range(n_components):
            obs_i = observations[labels == i]
            means.append(np.mean(obs_i, axis=0))
            covars.append(np.var(obs_i, axis=0) + 1e-2)
        model.startprob_ = np.ones(n_components) / n_components
        model.transmat_ = np.ones((n_components, n_components)) / n_components
        model.means_ = np.array(means)
        model.covars_ = np.array(covars)
        model.init_params = ''

    model.fit(observations)
    pred_states = model.predict(observations)
    return model, pred_states

In [27]:
def run_mle_default(observations, seed=None):
    return run_mle(observations, init_type='default', seed=seed)


def run_mle_kmeans(observations, seed=None):
    return run_mle(observations, init_type='kmeans', seed=seed)


def train_hmm_single_asset_default(series, n_components=2, random_state=42):
    X = series.values.reshape(-1, 1)
    model, _ = run_mle_default(X, seed=random_state)
    return model


def train_hmm_single_asset_kmeans(series, n_components=2, random_state=42):
    X = series.values.reshape(-1, 1)
    model, _ = run_mle_kmeans(X, seed=random_state)
    return model

#### 3.2 Training Sparse Jump model with max_feats=9 and lambda=40
##### 3.2.1 Defining feature selection framework

In [28]:
def compute_temporal_features_1d(y, window_len):
    T = len(y)
    feats = np.zeros((T, 9))
    half = (window_len - 1) // 2

    for t in range(T):
        feats[t, 0] = y[t]  # current
        feats[t, 1] = abs(y[t] - y[t - 1]) if t > 0 else 0.0
        feats[t, 2] = abs(y[t + 1] - y[t]) if t < T - 1 else 0.0

        left_c = max(0, t - half)
        right_c = min(T, t + half + 1)
        window_c = y[left_c:right_c]
        feats[t, 3] = np.mean(window_c)
        feats[t, 4] = np.std(window_c)

        left_l = max(0, t - window_len)
        window_l = y[left_l:t]
        feats[t, 5] = np.mean(window_l) if len(window_l) > 0 else 0.0
        feats[t, 6] = np.std(window_l) if len(window_l) > 0 else 0.0

        window_r = y[t:t + window_len]
        feats[t, 7] = np.mean(window_r) if len(window_r) > 0 else 0.0
        feats[t, 8] = np.std(window_r) if len(window_r) > 0 else 0.0

    return feats


def combine_features_1d(y, window_list=[5, 13]):
    feat_list = []
    for w in window_list:
        feat_list.append(compute_temporal_features_1d(y, w))
    return np.hstack(feat_list)

In [29]:
def train_sjm_single_asset(series, n_components=2, max_feats=9, lam=40.0, random_state=42):
    y = series.values
    X_raw = combine_features_1d(y)
    clipper = DataClipperStd(mul=3.0)
    scaler = StandardScalerPD()
    X_clipped = clipper.fit_transform(pd.DataFrame(X_raw))
    X_scaled = scaler.fit_transform(X_clipped)
    X_arr = X_scaled.values

    sjm = SparseJumpModel(
        n_components=n_components,
        max_feats=max_feats,
        jump_penalty=lam,
        cont=False,
        max_iter=20,
        random_state=random_state
    )
    sjm.fit(X_arr)
    return sjm

### 4.0 Allocation simulation

#### 4.1 Allocation workhorse functions
In this code we create the in which we fit the following models (each done in a seperate for loop such that we can store the relevant data such as return, weights, etc. in seperate dfs):
1. Equal weigted
2. Inverse volatility weighted
3. Mean-Variance-Optimal static portfolio
4. Hidden Markov Model Black Litterman where infered states are the identified regimes
5. Sparse Jump Model Black Litterman where infered states are the identified regimes

In [30]:
def backtest_portfolio(returns, weights):
    """
    Given a static weight vector 'weights' and a DataFrame of returns,
    compute the portfolio value series over time (starting from 1.0).
    """
    T = len(returns)
    portfolio_vals = np.zeros(T)
    portfolio_vals[0] = 1.0
    for t in range(T - 1):
        ret_t = returns.iloc[t].values
        portfolio_vals[t + 1] = portfolio_vals[t] * (1.0 + np.dot(weights, ret_t))
    return portfolio_vals


def equal_weight_allocation(n_assets):
    return np.ones(n_assets) / n_assets


def inverse_vol_weights(returns):
    stds = returns.std(axis=0).values + 1e-12
    w = 1.0 / stds
    return w / w.sum()


def static_mvo_allocation(returns):
    """
    Example static MVO using PyPortfolioOpt (just for illustration).
    """
    from pypfopt import expected_returns
    mu = pd.Series(CONST_RET, index=returns.columns)  # constant mu
    cov = risk_models.sample_cov(returns)

    ef = EfficientFrontier(mu, cov, weight_bounds=(0, 1), solver="SCS")
    ef_weights = ef.max_sharpe(risk_free_rate=RISK_FREE_RATE)
    return ef.clean_weights()


def black_litterman_allocation(view_vector, prior_cov):
    """
    Given a 'view_vector' (dict of {asset: expected return}) and a prior covariance,
    run Black-Litterman to obtain final weights.
    """
    pi = pd.Series(CONST_RET, index=prior_cov.columns)
    viewdict = {asset: v for asset, v in zip(ASSETS, view_vector)}

    bl = BlackLittermanModel(
        cov_matrix=prior_cov,
        pi=pi,
        absolute_views=viewdict,
        tau=BL_TAU,
        risk_aversion=1
    )
    bl_rets = bl.bl_returns()  
    bl_cov = bl.bl_cov()

    ef = EfficientFrontier(bl_rets, bl_cov, weight_bounds=(0, 1), solver="SCS")
    if max(bl_rets) <= RISK_FREE_RATE:
        ef_weights = ef.min_volatility()
    else:
        ef_weights = ef.max_sharpe(risk_free_rate=RISK_FREE_RATE)

    clean_weights = ef.clean_weights()
    w_array = np.array([clean_weights[a] for a in prior_cov.columns])
    return w_array

### 5.0 Performance metric evaluation:
Here we divide the performance metric into. We assume 250 data points to be 1 year off trading:
1. Return-Based Metrics 

Annualized Return: Average return per year. 

Cumulative Return: Total portfolio growth over time. 

2. Risk-Based Metrics 

Volatility: Standard deviation of returns. 

Downside Deviation: Measures negative return fluctuations. 

Max Drawdown (MDD): Largest portfolio decline from peak to trough. 

3. Risk-Adjusted Metrics 

Sharpe Ratio: Return per unit of total risk. 

Sortino Ratio: Return per unit of downside risk. 

Calmar Ratio: Return relative to max drawdown. 

4. Portfolio Stability & Adaptation 

Turnover Rate: Measures frequency of asset reallocation. 


We further split the performance three seperate tables with 1-state process, 2-state process, 3-state process




In [31]:
def compute_performance_metrics(portfolio_vals, weight_history=None, annual_factor=250):
    """
    Compute standard performance metrics for a given portfolio value series.
    """
    pv = np.asarray(portfolio_vals)
    rets = np.diff(pv) / pv[:-1]

    ann_ret = rets.mean() * annual_factor
    cum_ret = pv[-1] / pv[0] - 1
    ann_vol = rets.std() * np.sqrt(annual_factor)

    negative_rets = rets[rets < 0]
    ddev = negative_rets.std() * np.sqrt(annual_factor) if len(negative_rets) > 0 else 0.0
    max_dd = (pv / np.maximum.accumulate(pv) - 1).min()

    sharpe = ann_ret / (ann_vol + 1e-12)
    sortino = ann_ret / ddev if ddev > 1e-12 else np.nan
    calmar = ann_ret / abs(max_dd) if max_dd < 0 else np.nan

    if weight_history is not None and len(weight_history) > 1:
        turnover_list = []
        for t in range(1, len(weight_history)):
            turnover_list.append(np.sum(np.abs(weight_history[t] - weight_history[t - 1])))
        avg_turnover = np.mean(turnover_list)
    else:
        avg_turnover = 0.0

    return {
        "Annualized Return": ann_ret,
        "Cumulative Return": cum_ret,
        "Volatility": ann_vol,
        "Downside Deviation": ddev,
        "Max Drawdown": max_dd,
        "Sharpe Ratio": sharpe,
        "Sortino Ratio": sortino,
        "Calmar Ratio": calmar,
        "Turnover Rate": avg_turnover,
    }

#### 6.0 Helper function

In [32]:
def get_regime_means_single_asset(asset_series, regime_assignments):
    """
    For a single asset series and its assigned states, return a dictionary
    of {regime_label: mean_of_that_regime}.
    """
    unique_states = np.unique(regime_assignments)
    regime_means = {}
    for s in unique_states:
        regime_means[s] = asset_series[regime_assignments == s].mean()
    return regime_means

#### 7.0 Regime Based Asset Allocaiton

In [33]:
def regime_based_bl_backtest(df_test, states_test, regime_means_list, prior_cov, train_means_per_asset):
    """
    Daily rebalancing approach:
      For each day t, identify the regime for each asset i -> states_test[t,i].
      Then use the corresponding regime mean as the 'view' for that asset.
      Finally run black-litterman allocation to get daily weights.
    """
    T_test = len(df_test)
    portfolio_vals = np.zeros(T_test)
    portfolio_vals[0] = 1.0

    weight_history = np.zeros((T_test, N_ASSETS))

    # Start with some initial weight (e.g. equal)
    w_prev = equal_weight_allocation(N_ASSETS)
    weight_history[0] = w_prev

    # Step through each day
    for t in range(T_test - 1):
        view_vector = []
        for i in range(N_ASSETS):
            current_regime = states_test[t, i]
            # Just pick the regime-specific mean
            view_val = regime_means_list[i].get(current_regime, train_means_per_asset[i])
            view_vector.append(view_val)

        # Generate new weights from BL
        w_bl = black_litterman_allocation(view_vector, prior_cov)

        # Compute daily growth
        ret_t = df_test.iloc[t].values
        portfolio_vals[t + 1] = portfolio_vals[t] * (1.0 + np.dot(w_prev, ret_t))

        # Store new weight
        weight_history[t + 1] = w_bl
        w_prev = w_bl

    return portfolio_vals, weight_history

#### 8.0 Wrapper to run full allocation

In [34]:
def run_allocation(df):
    """
    Splits df into train/test. Trains HMM and SJM per asset.
    Then runs the 6 strategies:
      1) Equal Weight
      2) Inverse Vol
      3) Static MVO
      4) HMM-BL (Default)
      5) HMM-BL (Kmeans)
      6) SJM-BL
    Returns a dict of performance metrics for each strategy.
    """
    split_idx = int(len(df) * 0.8)
    df_train = df.iloc[:split_idx]
    df_test = df.iloc[split_idx:]
    prior_cov = df_train.cov()

    # -------------------------------
    # Train models per asset
    # -------------------------------
    hmm_models_default = []
    hmm_models_kmeans = []
    sjm_models = []

    hmm_states_default_train = np.zeros((split_idx, N_ASSETS), dtype=int)
    hmm_states_kmeans_train = np.zeros((split_idx, N_ASSETS), dtype=int)
    sjm_states_train = np.zeros((split_idx, N_ASSETS), dtype=int)

    for i, asset in enumerate(ASSETS):
        series_train = df_train[asset]

        # 1) HMM default
        hmm_mod_default = train_hmm_single_asset_default(series_train)
        hmm_mod_states_default = hmm_mod_default.predict(series_train.values.reshape(-1, 1))
        hmm_models_default.append(hmm_mod_default)
        hmm_states_default_train[:, i] = hmm_mod_states_default

        # 2) HMM kmeans
        hmm_mod_kmeans = train_hmm_single_asset_kmeans(series_train)
        hmm_mod_states_kmeans = hmm_mod_kmeans.predict(series_train.values.reshape(-1, 1))
        hmm_models_kmeans.append(hmm_mod_kmeans)
        hmm_states_kmeans_train[:, i] = hmm_mod_states_kmeans

        # 3) SJM
        sjm_mod = train_sjm_single_asset(series_train, n_components=2, lam=80.0)
        X_raw = combine_features_1d(series_train.values)
        clipper = DataClipperStd(mul=3.0)
        scaler = StandardScalerPD()
        X_test_clipped = clipper.fit_transform(pd.DataFrame(X_raw))
        X_test_scaled = scaler.fit_transform(X_test_clipped)
        sjm_states = sjm_mod.predict(X_test_scaled.values)
        sjm_models.append(sjm_mod)
        sjm_states_train[:, i] = sjm_states

    # -------------------------------
    # Compute in-sample regime means
    # -------------------------------
    hmm_regime_means_default = []
    hmm_regime_means_kmeans = []
    sjm_regime_means = []
    train_means_per_asset = []

    for i in range(N_ASSETS):
        asset_train = df_train.iloc[:, i]
        train_means_per_asset.append(asset_train.mean())

        hmm_regime_means_default.append(
            get_regime_means_single_asset(asset_train, hmm_states_default_train[:, i])
        )
        hmm_regime_means_kmeans.append(
            get_regime_means_single_asset(asset_train, hmm_states_kmeans_train[:, i])
        )
        sjm_regime_means.append(
            get_regime_means_single_asset(asset_train, sjm_states_train[:, i])
        )

    # -------------------------------
    # Predict test states
    # -------------------------------
    T_test = len(df_test)
    hmm_states_default_test = np.zeros((T_test, N_ASSETS), dtype=int)
    hmm_states_kmeans_test = np.zeros((T_test, N_ASSETS), dtype=int)
    sjm_states_test = np.zeros((T_test, N_ASSETS), dtype=int)

    for i, asset in enumerate(ASSETS):
        asset_series_test = df_test[asset].values.reshape(-1, 1)
        hmm_states_default_test[:, i] = hmm_models_default[i].predict(asset_series_test)
        hmm_states_kmeans_test[:, i] = hmm_models_kmeans[i].predict(asset_series_test)

        X_test_raw = combine_features_1d(df_test[asset].values)
        clipper = DataClipperStd(mul=3.0)
        scaler = StandardScalerPD()
        X_test_clipped = clipper.fit_transform(pd.DataFrame(X_test_raw))
        X_test_scaled = scaler.fit_transform(X_test_clipped)
        sjm_states_test[:, i] = sjm_models[i].predict(X_test_scaled.values)

    # -------------------------------
    # 6 Strategies
    # -------------------------------
    # 1) Equal-Weight
    w_ew = equal_weight_allocation(N_ASSETS)
    pv_ew = backtest_portfolio(df_test, w_ew)
    w_hist_ew = np.tile(w_ew, (T_test, 1))

    # 2) Inverse Vol
    w_iv = inverse_vol_weights(df_test)
    pv_iv = backtest_portfolio(df_test, w_iv)
    w_hist_iv = np.tile(w_iv, (T_test, 1))

    # 3) Static MVO
    w_mvo_dict = static_mvo_allocation(df_train)
    w_mvo_arr = np.array([w_mvo_dict[a] for a in ASSETS])
    pv_mvo = backtest_portfolio(df_test, w_mvo_arr)
    w_hist_mvo = np.tile(w_mvo_arr, (T_test, 1))

    # 4) HMM-BL (Default)
    pv_hmmbl_default, w_hmmbl_default = regime_based_bl_backtest(
        df_test, hmm_states_default_test,
        hmm_regime_means_default,
        prior_cov,
        train_means_per_asset
    )

    # 5) HMM-BL (K-Means)
    pv_hmmbl_kmeans, w_hmmbl_kmeans = regime_based_bl_backtest(
        df_test, hmm_states_kmeans_test,
        hmm_regime_means_kmeans,
        prior_cov,
        train_means_per_asset
    )

    # 6) SJM-BL
    pv_sjmbl, w_sjmbl = regime_based_bl_backtest(
        df_test, sjm_states_test,
        sjm_regime_means,
        prior_cov,
        train_means_per_asset
    )

    # -------------------------------
    # Performance for each strategy
    # -------------------------------
    perf = {
        "EW": compute_performance_metrics(pv_ew, w_hist_ew),
        "IV": compute_performance_metrics(pv_iv, w_hist_iv),
        "MVO": compute_performance_metrics(pv_mvo, w_hist_mvo),
        "HMM-BL-Default": compute_performance_metrics(pv_hmmbl_default, w_hmmbl_default),
        "HMM-BL-KMeans": compute_performance_metrics(pv_hmmbl_kmeans, w_hmmbl_kmeans),
        "SJM-BL": compute_performance_metrics(pv_sjmbl, w_sjmbl)
    }

    return perf

#### 9.0 Full scenario: 1-state, 2-state, 3-state runs



In [35]:
def run_scenario_123(T_sim=1000, seed1=None, seed2=None, seed3=None):
    """
    Simulate & run 1-state, 2-state, 3-state data sets.
    We pass different seeds for each scenario so each is truly random.
    """
    # 1-state
    df1_full = simulate_1state_data(T_sim, seed=seed1)
    perf_1 = run_allocation(df1_full)

    # 2-state
    df2_full, _ = simulate_2state_data(T_sim, seed=seed2)
    perf_2 = run_allocation(df2_full)

    # 3-state
    df3_full, _ = simulate_3state_data(T_sim, seed=seed3)
    perf_3 = run_allocation(df3_full)

    return {
        "1state": perf_1,
        "2state": perf_2,
        "3state": perf_3
    }


def single_monte_carlo_run(run_id, T_sim=1000):
    """
    A single replication of the full scenario: 1-state, 2-state, and 3-state study.
    Each scenario uses a unique seed for variety.
    """
    print(f"Running simulation {run_id}...")

    # Example: generate 3 different seeds for the 3 scenarios
    # so each scenario draws different random data on each run.
    seed_for_1state = run_id * 1000 + 11
    seed_for_2state = run_id * 1000 + 22
    seed_for_3state = run_id * 1000 + 33

    results = run_scenario_123(
        T_sim=T_sim,
        seed1=seed_for_1state,
        seed2=seed_for_2state,
        seed3=seed_for_3state
    )
    return results


def run_monte_carlo_study(n_runs=10, T_sim=1000):
    """
    Run n_runs Monte Carlo replications in parallel. Collect all performance metrics
    for each scenario & strategy, then do Wilcoxon tests on Sharpe Ratios.
    """
    n_cores = multiprocessing.cpu_count()
    print(f"detected {n_cores} cores")

    # Parallel or single-thread as desired. If you want fewer cores, set n_jobs accordingly.
    all_results = Parallel(n_jobs=n_cores)(
        delayed(single_monte_carlo_run)(i + 1, T_sim) for i in range(n_runs)
    )

    # Strategies we compare
    strategies = ["EW", "IV", "MVO", "HMM-BL-Default", "HMM-BL-KMeans", "SJM-BL"]
    scenarios = ["1state", "2state", "3state"]

    # -----------------------------------
    #  Collect Sharpe for Wilcoxon
    # -----------------------------------
    sharpe_data = {sc: {st: [] for st in strategies} for sc in scenarios}

    # Also store entire metric distributions for average stats
    # We'll do e.g. metric_data[sc][st]["Annualized Return"] = [run1, run2, ...]
    all_metrics = {}
    for sc in scenarios:
        all_metrics[sc] = {}
        for st in strategies:
            all_metrics[sc][st] = {
                "Annualized Return": [],
                "Cumulative Return": [],
                "Volatility": [],
                "Downside Deviation": [],
                "Max Drawdown": [],
                "Sharpe Ratio": [],
                "Sortino Ratio": [],
                "Calmar Ratio": [],
                "Turnover Rate": [],
            }

    for run_res in all_results:
        # run_res is a dict: {"1state": {...}, "2state": {...}, "3state": {...}}
        for sc in scenarios:
            for st in strategies:
                # Pull out the dictionary of metrics for that scenario & strategy
                metrics_dict = run_res[sc][st]
                # Keep track of Sharpe for Wilcoxon
                sharpe_data[sc][st].append(metrics_dict["Sharpe Ratio"])
                # Keep track of all metrics
                for mkey in all_metrics[sc][st]:
                    all_metrics[sc][st][mkey].append(metrics_dict[mkey])

    # -----------------------------------
    #  Print/Collect Wilcoxon results
    # -----------------------------------
    print("\n==== Wilcoxon Tests (SJM-BL vs. others, Sharpe Ratio) ====")
    wilcoxon_rows = []
    for sc in scenarios:
        sjm_sharpes = sharpe_data[sc]["SJM-BL"]
        for st in strategies:
            if st == "SJM-BL":
                continue
            other_sharpes = sharpe_data[sc][st]
            stat, pval = wilcoxon(sjm_sharpes, other_sharpes, alternative='two-sided')
            print(f"{sc} | SJM-BL vs {st}: Wilcoxon stat={stat:.4f}, p={pval:.4g}")

            wilcoxon_rows.append({
                "Scenario": sc,
                "Comparison": f"SJM-BL vs {st}",
                "Statistic": stat,
                "p-value": pval
            })

    df_wilcoxon = pd.DataFrame(wilcoxon_rows)
    print("\nWilcoxon Results Table:")
    print(df_wilcoxon.to_string(index=False))

    # -----------------------------------
    #  Compute & Print Average Metrics
    # -----------------------------------
    print("\n==== Average Performance Metrics (across all runs) ====")
    for sc in scenarios:
        rows = []
        for st in strategies:
            # compute mean of each metric across runs
            metric_means = {}
            for mkey, vals in all_metrics[sc][st].items():
                metric_means[mkey] = np.mean(vals)
            row = {"Strategy": st}
            row.update(metric_means)
            rows.append(row)

        df_avg = pd.DataFrame(rows)
        df_avg.set_index("Strategy", inplace=True)
        print(f"\n--- {sc.upper()} ---")
        print(df_avg.to_string())

    return sharpe_data, all_metrics, all_results, df_wilcoxon

### 10.0 Main execution: Run simulation and output performance metrics

In [36]:
if __name__ == "__main__":
    # Example: run 5 replications
    n_simulations = 100
    T_sim = 10000

    # Run parallel simulation
    sharpe_data, all_metrics, all_runs, df_wilcoxon = run_monte_carlo_study(
        n_runs=n_simulations,
        T_sim=T_sim
    )

detected 16 cores

==== Wilcoxon Tests (SJM-BL vs. others, Sharpe Ratio) ====
1state | SJM-BL vs EW: Wilcoxon stat=266.0000, p=8.026e-15
1state | SJM-BL vs IV: Wilcoxon stat=268.0000, p=8.473e-15
1state | SJM-BL vs MVO: Wilcoxon stat=96.0000, p=6.729e-17
1state | SJM-BL vs HMM-BL-Default: Wilcoxon stat=228.0000, p=2.838e-15
1state | SJM-BL vs HMM-BL-KMeans: Wilcoxon stat=31.0000, p=9.893e-18
2state | SJM-BL vs EW: Wilcoxon stat=1908.0000, p=0.03388
2state | SJM-BL vs IV: Wilcoxon stat=1877.0000, p=0.02588
2state | SJM-BL vs MVO: Wilcoxon stat=1044.0000, p=3.54e-07
2state | SJM-BL vs HMM-BL-Default: Wilcoxon stat=2091.0000, p=0.1356
2state | SJM-BL vs HMM-BL-KMeans: Wilcoxon stat=1707.0000, p=0.004915
3state | SJM-BL vs EW: Wilcoxon stat=1398.0000, p=0.0001066
3state | SJM-BL vs IV: Wilcoxon stat=1418.0000, p=0.0001411
3state | SJM-BL vs MVO: Wilcoxon stat=611.0000, p=4.674e-11
3state | SJM-BL vs HMM-BL-Default: Wilcoxon stat=2337.0000, p=0.518
3state | SJM-BL vs HMM-BL-KMeans: Wilcoxon

### 8.0 Visualizations